In [1]:
import numpy as np
from scipy.signal import convolve2d 
from os import path, getcwd, system, getenv
from astropy.io import fits
from time import perf_counter
import urllib.request
import os

import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import glob
import matplotlib.gridspec as gridspec

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.backends.backend_pdf import PdfPages

import sys,inspect
currentdir = path.dirname(path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = path.dirname(currentdir)
sys.path.append(parentdir)
# from deconv import cube_deconv
# from ppxf_wrap import ppxf_wrap

# import warnings

# if True:
#     print('Set warnings.filterwarnings as "ignore"')
#     print('Warning Message will not be printed')
#     warnings.filterwarnings("ignore")

from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import constants
import pandas as pd

# import plotly.express as px
from astroquery.mast import Observations


from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.nddata.utils import Cutout2D

from astropy.wcs import WCS
import astropy

from urllib.error import HTTPError
from astroquery.skyview import SkyView
import pickle

maindir = "C:/Users/maxri/Desktop/Classes 4-1/Research/fuse"

In [174]:
#Create Object List/
#Save to Object Tables

catalogdir = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VI_129_fuse.dat.fits"
catalog = fits.open(catalogdir)

catheader = catalog[0]; catlist = catalog[1];

dflist = pd.DataFrame(catlist.data)
catalog.close()


catalogdir2 = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VII_1B_catalog.dat.fits"
catalog2 = fits.open(catalogdir2)

catheader2 = catalog2[0]; catlist2 = catalog2[1];

dflist2 = pd.DataFrame(catlist2.data)
catalog2.close()
dflist2

#Loop
for ii in range(90,91,1):
    
    computer = 1

    obn = 89
    obn = ii
    skydowni ="./images_redOb{}".format(obn); 





    obs=dflist.query("Class == {} and Avail=='y'".format(obn));

    dflist



    obsn1 = []
    for i in obs.iterrows():
        b = [i[1][1].split()[0],i[1][2], i[1][3]] 
        obsn1.append(b)



    obsn1

    s1 = maindir+"/tbls/Objects{}.tbl".format(obn); 

    # #Reading Index
    # indexdir = f"{maindir}/tbls/{obn}index.txt"
    # indexdir

    # ipd = pd.read_table(indexdir, header = None)

    obsn1



    minsep = 0.5
    pairs = []
    for i in range(len(obsn1)):
    #     if ipd[0][i] == 0:
    #         continue
        n1 =obsn1[i][0]; ra1 = obsn1[i][1]; dec1 = obsn1[i][2]
        c1 = SkyCoord(ra1,dec1, unit=u.deg)
        c4 = []; c5 = []; pc = 0;
        c4.append([f"Fuse{n1}",ra1,dec1]);
        for j in dflist2.iterrows():
            ra2 = j[1][3]*15 + j[1][4]/4; 
            if j[1][5] =="-":
                dec2 = (j[1][6]+j[1][7]/60) * (-1)
            else:
                dec2 = j[1][6]+j[1][7]/60
            c2 = SkyCoord(ra2,dec2, unit = u.deg)
            if c1.separation(c2).deg < minsep:
                c5.append([f"NGC-NGC{j[1][0]}",ra2,dec2])
                print("Pair found"); pc = pc+1
        b = c4[0]
        if pc == 0:
            b = b + ["NONE"]
        if pc == 1:
            b = c4[0] + c5[0]
        if pc >1:
            for j5 in range(pc):
                b = b + c5[j5]
        pairs.append(b)

    pairs

    pairstring = maindir + f"/corrtbls/corr{obn}.txt"

    with open(pairstring, 'w') as f:
        for i in range(len(pairs)):
            for j in range(len(pairs[i])):
                f.write(f"{pairs[i][j]}"); f.write(" ")
            f.write('\n')
        f.close()

Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found
Pair found


In [171]:
cut_widths=np.array([
            240,240,240,240,
            240,240,240,240,
            240,240,240,240,
            240, 240,240,240,
        ])/3600*3 
cut_widths[1]

0.2

['FuseNGC1705', 73.5562, -53.3609, 'NGC-NGC1705', 73.425, -53.38333333333333]

2

2

In [ ]:
]